In [25]:
import pickle
import warnings
import pandas as pd
import numpy as np

from datetime import datetime, timezone
from ecoselekt.settings import settings

warnings.filterwarnings('ignore')

In [20]:
delta_df = pd.DataFrame(columns=["project", "windowi", "windowj", "months"])

for project_name in settings.PROJECTS:
    with open(settings.DATA_DIR / f"{settings.EXP_ID}_{project_name}_windows.pkl", "rb") as f:
        windows = pickle.load(f)
    print(f"{project_name}: {len(windows)}")
    for windowi in range(len(windows)-1):
        for windowj in range(windowi+1, len(windows)):
            x = datetime.fromtimestamp(windows[windowi]["author_date"].min(), tz=timezone.utc)
            y = datetime.fromtimestamp(windows[windowj]["author_date"].max(), tz=timezone.utc)
            delta_df = delta_df.append(
                {
                    "project": project_name,
                    "windowi": windowi,
                    "windowj": windowj,
                    "months": (y-x).days//30
                },
                ignore_index=True
            )

delta_df.head()

activemq: 26
camel: 109
cassandra: 36
flink: 54
groovy: 36
hbase: 39
hive: 30
ignite: 56


,project,windowi,windowj,months
0,activemq,0,1,12
1,activemq,0,2,15
2,activemq,0,3,21
3,activemq,0,4,26
4,activemq,0,5,28


In [21]:
delta_df[(delta_df["project"] == "camel") & (delta_df["windowi"] == 15) & (delta_df["windowj"] == 86)]

,project,windowi,windowj,months
1910,camel,15,86,102


In [27]:
months = []
for project_name in settings.PROJECTS:
    with open(settings.DATA_DIR / f"{settings.EXP_ID}_{project_name}_windows.pkl", "rb") as f:
        windows = pickle.load(f)
    for i in range(1, len(windows)-15):
        x = datetime.fromtimestamp(windows[i]["author_date"].min(), tz=timezone.utc)
        y = datetime.fromtimestamp(windows[i+15]["author_date"].max(), tz=timezone.utc)
        months.append((y-x).days//30)
print(f"min: {min(months)} max: {max(months)} median: {np.median(months)} mean: {np.mean(months)}")

min: 8 max: 123 median: 33.0 mean: 41.89922480620155


In [32]:
days = []

for project_name in settings.PROJECTS:
    with open(settings.DATA_DIR / f"{settings.EXP_ID}_{project_name}_windows.pkl", "rb") as f:
        windows = pickle.load(f)
    for window in windows:
        x = datetime.fromtimestamp(window["author_date"].min(), tz=timezone.utc)
        y = datetime.fromtimestamp(window["author_date"].max(), tz=timezone.utc)
        days.append((y-x).days)

print(f"min: {min(days)} max: {max(days)} median: {np.median(days)} mean: {np.mean(days)}")

min: 27 max: 1561 median: 289.5 mean: 365.7383419689119
